# Лабораторная работа 6. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы Поляков Вадим Дмитриевич БВТ2002
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



### Выполнение:

In [2]:
import math
import time
import networkx as nx
import ipywidgets as widgets
import pandas as pd
from ipywidgets import interact, interact_manual
from pyvis.network import Network
from collections import deque

def read_matrix(file):
    matrix = []
    with open("./" + file, "r") as f:
        for line in f.readlines():
            matrix.append(line.split())
    matrix = [[int(num) for num in matrix[i]] for i in range(len(matrix))]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 0 and i != j:
                matrix[i][j] = math.inf
    return matrix

def make_graph(matrix, m_type):
    G = nx.MultiDiGraph()
    if m_type == 0:
        for i in range(len(matrix[0])):
            x = -1
            for j in range(len(matrix)):
                if x != -1 and matrix[j][i] != 0 and matrix[j][i] != math.inf:
                    G.add_edge(x, j+1, color='red')
                if x == -1 and matrix[j][i] != 0 and matrix[j][i] != math.inf:
                    x = j+1
    if m_type == 1:
        for i in range(len(matrix)):
            G.add_node(i+1, color='red')
            for j in range(len(matrix[0])):
                if i != j and matrix[i][j] != math.inf:
                    G.add_edge(j+1, i+1, weight=matrix[i][j], color='red')
    nt = Network(notebook=True, directed=True)
    nt.from_nx(G)
    return nt

def user_make_graph(add_node="", add_edge="", edge_width="1"):
    if add_node != "":
        add_node = add_node.split()
        graph.add_node(int(add_node[0]), color='red')
    if add_edge != "":
        if "," in add_edge and edge_width != "":
            graph.add_edge(int(add_edge.split(",")[0]), int(add_edge.split(",")[1]), width=edge_width, weight=edge_width, color='red')
    nb.from_nx(graph)
    return nb.show("basic.html")

def make_adjacency_matrix(G):
    matrix = [[0]*len(G.nodes) for i in range(len(G.nodes))]
    for i in range(1,len(G.nodes)):
        pairs = list(G.edges)
        for j in range(len(pairs)):
            if pairs[j]['from'] == i:
                matrix[pairs[j]['to']-1][i-1] = pairs[j]['weight']
            if pairs[j]['to'] == i:
                matrix[i-1][pairs[j]['from']-1] = pairs[j]['weight']
    return matrix

def new_graph(G, res):
    graph = nx.MultiDiGraph()
    edges = list(G.edges)
    for i in range(len(edges)):
        if not edges[i]['from'] in res or not edges[i]['to'] in res:
            graph.add_edge(edges[i]['from'], edges[i]['to'], weight=edges[i]['weight'], width=1, color="red")
    for i in range(len(edges)):
        if edges[i]['from'] in res and edges[i]['to'] in res:
            graph.add_edge(edges[i]['from'], edges[i]['to'], weight=edges[i]['weight'], width=3, color="green")
    nt = Network(notebook=True, directed=True)
    nt.from_nx(graph)
    return nt

def remove_edge(G, edge):
    M = make_adjacency_matrix(G)
    M[edge[1]-1][edge[0]-1] = 0
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] == 0 and i != j:
                M[i][j] = math.inf
    G = make_graph(M, 1)
    return G

In [3]:
#Алгоритм Флойда-Уоршелла
#Работает корректно, если в графе нет циклов отрицательной величины, 
#а в случае, когда такой цикл есть, позволяет найти хотя бы один такой цикл

#Сложность
#O(|V^3|")

#Находит кратчайший путь от всех пар вершин
#Негативные веса в ребрах разрешаются

#заведомо нет отрицательных циклов

def floyd_warshall(G, start, end):
    start -= 1
    end -= 1
    W = make_adjacency_matrix(G)
    for i in range(len(W)):
        for j in range(len(W[i])):
            if W[i][j] == 0 and i != j:
                W[i][j] = math.inf
    n = len(W)
    A = [[W[i][j] for i in range(n)] for j in range(n)] 
    V = [[math.inf for i in range(n)] for j in range(n)]
    for k in range(n): 
        for i in range(n):
            for j in range(n): 
                if A[i][j] > (A[i][k] + A[k][j]):
                    A[i][j] = A[i][k] + A[k][j]
                    V[i][j] = k+1
    path = [end+1, V[start][end]]
    if path[-1] != math.inf:
        while end != start+1:
            end = V[start][end]-1
            path.append(end)
    else:
        path.pop()
        path.append(start+1)
    return path[::-1]

In [5]:
G = make_graph(read_matrix("adjacency_matrix2.txt"), 1)

res = floyd_warshall(G, 1, 6)
print(res)

new_graph(G, res).show("basic.html")

[1, 2, 3, 4, 6]


In [ ]:
#Алгоритм Дейкстры
#Сложность O(V^2 + Е)

#Находит кратчайшее расстояние от выбранной ноды до всех остальных
#Отрицательные веса в ребрах не разрешаются!

def dijkstra(G, start, end):
    start -= 1
    end -= 1
    w = make_adjacency_matrix(G)
    for i in range(len(w)):
        for j in range(len(w[i])):
            if w[i][j] == 0 and i != j:
                w[i][j] = math.inf
    n = len(w)
    dist = [math.inf] * n
    dist[start] = 0
    prev = [None] * n
    used = [False] * n
    min_dist = 0
    min_vertex = start
    while min_dist < math.inf:
        i = min_vertex 
        used[i] = True 
        for j in range(n): 
            if dist[i] + w[j][i] < dist[j]: 
                dist[j] = dist[i] + w[j][i] 
                prev[j] = i+1
        min_dist = math.inf
        for j in range(n):
            if not used[j] and dist[j] < min_dist:
                min_dist = dist[j]
                min_vertex = j
    path = [end+1]
    while end != start:
        end = prev[end]-1
        path.append(end+1)
    return path[::-1]

In [ ]:
G = make_graph(read_matrix("adjacency_matrix2.txt"), 1)
res = dijkstra(G, 1, 6)
print(res)
new_graph(G, res).show("basic.html")

In [ ]:
#Алгоритм Беллмана-Форда
#заведомо нет отрицательных циклов
#Худшее время O(VE)
#Лучшее время O(E)

#Находит кратчайшее расстояние от одной ноды до всех остальных 
#Негативные веса в ребрах разрешаются!

def bellman_ford(G, start, end):
    start -= 1
    end -= 1
    W = make_adjacency_matrix(G)
    for i in range(len(W)):
        for j in range(len(W[i])):
            if W[i][j] == 0 and i != j:
                W[i][j] = math.inf
    N = len(W)
    F = [[math.inf] * N for i in range(N)]
    V = [[math.inf] * N for i in range(N)]
    F[0][start] = 0 
    
    for k in range(1, N): 
        for i in range(N): 
            F[k][i] = F[k - 1][i]
            for j in range(N):
                if F[k - 1][j] + W[i][j] < F[k][i]:
                    V[k][i] = j+1
                    
    for i in range(max(len(F), len(V))-1, 0, -1):
        if F[i] == F[i-1]:
            F.pop(i)
        if V[i] == V[i-1]:
            V.pop(i)
    V.pop()
    P = [end+1]
    for i in range(len(V)):
        if V[i][end] != math.inf:
            x = i
            break
    while end != start+1:
        end = V[x][P[-1]-1]
        x -= 1
        P.append(end)
    if len(P) == 1:
        P.append(start+1)
    return P[::-1]

In [ ]:
G = make_graph(read_matrix("adjacency_matrix2.txt"), 1)
res = bellman_ford(G, 1, 6)
print(res)
new_graph(G, res).show("basic.html")

In [ ]:
#Алгоритм Джонсона
#позволяет найти кратчайшие пути между всеми парами вершин взвешенного ориентированного графа. 
#В графе содержатся рёбра с положительным или отрицательным весом, 
#но отсутствуют циклы с отрицательным весом

#Сложность O(V*E*lnV)

def bellman_ford_len(W, start, end):
    N = len(W)
    F = [[math.inf] * N for i in range(N)]
    F[0][start-1] = 0 
    for k in range(1, N): 
        for i in range(N): 
            F[k][i] = F[k - 1][i]
            for j in range(N):
                if F[k - 1][j] + W[i][j] < F[k][i]:
                    F[k][i] = F[k - 1][j] + W[i][j]
    return F[-1][end-1]

def johnson(G, start, end):
    D = make_adjacency_matrix(G)
    for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] == 0 and i != j:
                    D[i][j] = math.inf
    negatives = 0
    for i in range(len(D)):
        for j in range(len(D[i])):
            if D[i][j] < 0:
                negatives = 1
    if negatives != 0:
        D.append([math.inf for i in range(len(D))])
        for i in range(len(D)):
            D[i].append(0)
        nodes = []
        for i in range(len(G.nodes)):
            nodes.append(list(G.nodes)[i]['id'])
        nodes.sort()
        lens = []
        for i in range(len(nodes)):
            lens.append(bellman_ford_len(D, len(D), nodes[i]))
        for i in range(len(D)):
            D[i].pop()
        D.pop()
        for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] != 0 and D[i][j] != math.inf:
                    D[i][j] = D[i][j] + lens[j] - lens[i]
        for i in range(len(D)):
            for j in range(len(D[i])):
                if D[i][j] != math.inf and i != j:
                    D[i][j] += 1
    G = make_graph(D, 1)
    res = dijkstra(G, start, end)
    return res

In [ ]:
G = make_graph(read_matrix("adjacency_matrix_neg.txt"), 1)
res = johnson(G, 1, 6)
print(res)
new_graph(G, res).show("basic.html")

In [1]:
def levit(G, start, end):
    start -= 1
    end -= 1
    N = len(G.nodes)
    D = [math.inf for i in range(N)]
    D[start] = 0
    cache = deque([start])
    state = [2 for i in range(N)]
    state[start] = 1
    P = [-1 for i in range(N)]
    while cache:
        vertex = cache.popleft()
        state[vertex] = 0
        for i in range(len(G.neighbors(vertex+1))):
            neigh = list(G.neighbors(vertex+1))[i]-1
            for j in range(len(G.edges)):
                if list(G.edges)[j]['from'] == vertex+1 and list(G.edges)[j]['to'] == neigh+1:
                    weight = G.edges[j]['weight']
            if D[vertex] + weight < D[neigh]:
                D[neigh] = D[vertex] + weight
                if state[neigh] == 2:
                    cache.append(neigh)
                if state[neigh] == 0:
                    cache.appendleft(neigh)
                P[neigh] = vertex
                state[neigh] = 1
    path = []
    vertex = end
    while vertex != -1:
        path.append(vertex+1)
        vertex = P[vertex]
    return path[::-1]

In [ ]:
G = make_graph(read_matrix("adjacency_matrix2.txt"), 1)
res = levit(G, 1, 6)
print(res)
new_graph(G, res).show("basic.html")

In [ ]:
#Сложность алгоритма
#O(V^2)

#Работает в графах с положительными значениями
#Находит все пути до необходимой ноды
#Сначала находит самый оптимальный - затем по убыванию

def yen(G, start, end, total_paths = 2):
    start -= 1
    end -= 1
    paths = [[] for i in range(total_paths)]
    paths[0] = dijkstra(G, start+1, end+1)
    
    for i in range(1,len(paths)):
        edges = []
        
        for j in range(len(G.edges)):
            edges.append([list(G.edges)[j]['from'], list(G.edges)[j]['to']])
        length = 0
        
        for j in range(len(G.edges)):
            if list(G.edges)[j]['from'] == paths[i-1][0] and list(G.edges)[j]['to'] == paths[i-1][1]:
                length = list(G.edges)[j]['weight']
                
        G = remove_edge(G, [paths[i-1][0], paths[i-1][1]])
        paths[i] = dijkstra(G, start+1, end+1)
        G.add_edge(paths[i-1][0], paths[i-1][1], weight=length)
        
    return paths

In [ ]:
G = make_graph(read_matrix("adjacency_matrix_yen.txt"), 1)
res = yen(G, 1, 6)
print(res)
new_graph(G, res[0]).show("basic.html")

In [5]:
print("Алгоритм Флойда-Уоршелла:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        floyd_warshall(G, 1, i)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

Алгоритм Флойда-Уоршелла:
6 ребер: 0.997 миллисекунд
7 ребер: 0.997 миллисекунд
8 ребер: 0.997 миллисекунд


In [ ]:
print("Алгоритм Дейкстры:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        dijkstra(G, 1, i)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

In [ ]:
print("Алгоритм Беллмана-Форда:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        bellman_ford(G, 1, i)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

In [ ]:
print("Алгоритм Джонсона:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        johnson(G, 1, i)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

In [ ]:
print("Алгоритм Левита:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        levit(G, 1, i)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

In [ ]:
print("Алгоритм Йена:")

for i in range(6,9):
    m = read_matrix("adjacency_matrix_"+str(i)+".txt")
    G = make_graph(m, 1)
    while True:
        prev = time.time()
        yen(G, 1, i, 1)
        if time.time()-prev != 0:
            break
    print(str(i) +" ребер: "+str(round((time.time()-prev)*1000000)/1000)+" миллисекунд")

### Вывод